In [1]:
%load_ext autoreload 
%autoreload 2

import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(os.path.join(os.getcwd(), ".."))

import math
import numpy as np
import pandas as pd
from numpy import array
from numpy import array, arange, abs as np_abs
from numpy.fft import rfft, rfftfreq
from math import sin, pi
from scipy import signal
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import random

import importlib
import model as md
import view as vw

importlib.reload(md)
importlib.reload(vw)

c:\Users\Артем\Desktop\Вуз\Аспирантура\Диссертация\Алгоритм\Relaxation_frequency_phase_algorithm\notebooks\Solo_ms\..


<module 'view' from 'c:\\Users\\Артем\\Desktop\\Вуз\\Аспирантура\\Диссертация\\Алгоритм\\Relaxation_frequency_phase_algorithm\\view\\__init__.py'>

In [3]:
from SALib.sample import sobol as sobol_sampler
from SALib.analyze import sobol as sobol_analyzer
import numpy as np

phase_method = md.get_phase_maxpoint_diff

problem = {
    'num_vars': 5,
    'names': ['F', 'fs_mult', 'duration_T', 'phase', 'SNR'],
    'bounds': [
        [300e3, 500e3],
        [3, 100],
        [3, 200],
        [1, 179],
        [20, 60]
    ]
}

param_values = sobol_sampler.sample(problem, N=2048)

Y = []
for p in param_values:
    F, fs_mult, duration_T, phase_real, SNR = p
    duration = duration_T / F

    t = np.arange(0, duration, 1/(fs_mult * F))
    U = md.generate_sin(t, F, 2)
    I = md.generate_sin(t, F, 2, phase_real)

    # добавляем шум
    noise_power = np.mean(U**2) / (10**(SNR / 10))
    noise = np.sqrt(noise_power) * np.random.randn(len(U))
    U += noise
    I += noise

    time, phase_mes = phase_method(t, U, I, F)
    phase_kde = md.get_kde_mode(phase_mes)
    phase_error = abs(phase_kde) - abs(phase_real)
    Y.append(abs(phase_error))

Y = np.array(Y)

Si = sobol_analyzer.analyze(problem, Y, calc_second_order=True, print_to_console=True)


                  ST   ST_conf
F           0.316545  0.054560
fs_mult     0.524340  0.090681
duration_T  0.404731  0.066344
phase       0.572415  0.097242
SNR         0.744024  0.111842
                  S1   S1_conf
F          -0.012026  0.032603
fs_mult     0.073445  0.049401
duration_T  0.009806  0.047555
phase       0.032503  0.046047
SNR         0.283532  0.061684
                             S2   S2_conf
(F, fs_mult)           0.014760  0.053094
(F, duration_T)        0.016852  0.054013
(F, phase)             0.055603  0.040030
(F, SNR)               0.006913  0.056741
(fs_mult, duration_T) -0.014985  0.071667
(fs_mult, phase)       0.004547  0.084929
(fs_mult, SNR)         0.020328  0.081383
(duration_T, phase)   -0.003074  0.060799
(duration_T, SNR)      0.038839  0.070606
(phase, SNR)           0.105305  0.103641
